In [2]:
pip install reportlab

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 5.2 MB/s  0:00:00 eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import glob
import os
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas

In [4]:
# Constants
EMOTION_LABELS = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
TRAIN_PATH = "/Users/nazmussakibsheam/GUB/Data-mining-lab/face_expression_detector/webcam_dataset/train"
EMOTION_HISTORY_PATH = 'emotion_history.csv'

In [5]:
def generate_pdf_report(df):
    """Generate a PDF report with emotion statistics and plots."""
    pdf_path = 'emotion_report.pdf'
    c = canvas.Canvas(pdf_path, pagesize=letter)
    c.setFont("Helvetica", 12)
    c.drawString(100, 750, "Facial Expression Recognition Report")
    
    # Emotion distribution
    c.drawString(100, 700, "Emotion Distribution:")
    emotion_counts = Counter(df['Emotion'])
    y_pos = 680
    for emo, count in emotion_counts.items():
        c.drawString(120, y_pos, f"{emo}: {count}")
        y_pos -= 20

    # Include plots
    if os.path.exists('confusion_matrix.png'):
        c.drawImage('confusion_matrix.png', 100, 400, width=400, height=300)
    if os.path.exists('training_history.png'):
        c.drawImage('training_history.png', 100, 100, width=400, height=200)
    
    c.save()
    return pdf_path

In [6]:
def streamlit_dashboard():
    """Launch Streamlit dashboard to visualize emotion trends."""
    st.title("Facial Expression Recognition Dashboard")
    st.write("Analyze emotions detected during the webcam session.")

    if not os.path.exists(EMOTION_HISTORY_PATH):
        st.error("No emotion history found. Run face_expression_detector.ipynb first.")
        return

    df = pd.read_csv(EMOTION_HISTORY_PATH)
    if df.empty:
        st.warning("No emotion data available.")
        return

    # Sort by timestamp to ensure monotonic index
    df['Timestamp'] = pd.to_datetime(df['Timestamp'])
    df = df.sort_values('Timestamp')

    # Emotion Timeline
    st.subheader("Emotion Timeline")
    df['TimeBin'] = df['Timestamp'].dt.floor('S')  # Bin by second
    timeline_data = df.groupby('TimeBin')['Emotion'].value_counts().unstack().fillna(0)
    st.line_chart(timeline_data)

    # Emotion Distribution
    st.subheader("Emotion Distribution")
    emotion_counts = Counter(df['Emotion'])
    st.bar_chart(emotion_counts)

    # Captured Images
    st.subheader("Captured Images")
    cols = st.columns(3)
    for i, img_path in enumerate(glob.glob(os.path.join(TRAIN_PATH, '*/*.jpg'))[:9]):
        with cols[i % 3]:
            st.image(img_path, caption=os.path.basename(os.path.dirname(img_path)))

    # Download CSV
    st.subheader("Download CSV Report")
    st.download_button("Download CSV", df.to_csv(index=False), "emotion_report.csv", "text/csv")

    # Download PDF
    st.subheader("Download PDF Report")
    pdf_path = generate_pdf_report(df)
    with open(pdf_path, "rb") as f:
        st.download_button("Download PDF", f, "emotion_report.pdf", "application/pdf")


In [7]:
if __name__ == "__main__":
    streamlit_dashboard()

2025-08-09 20:35:05.871 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-09 20:35:05.906 
  command:

    streamlit run /Users/nazmussakibsheam/Dev/Thesis/10 2/mp-env/lib/python3.10/site-packages/ipykernel_launcher.py [ARGUMENTS]
2025-08-09 20:35:05.906 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-09 20:35:05.907 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-09 20:35:05.907 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-09 20:35:05.907 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-09 20:35:05.908 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-09 20: